# Loading a pretrained model and evaluating it

In [1]:
# Import libraries and such
import numpy as np
#import tensorflow as tf
from keras.models import load_model
from sklearn.metrics import roc_auc_score, confusion_matrix, accuracy_score, auc
from sklearn import metrics

# Model Metric Function (When loading in pretrained models)
def auc_roc(y_true, y_pred):
    return tf.py_func(metrics.roc_auc_score, (y_true, y_pred), tf.double)

def print_cm(cm, labels, hide_zeroes=False, hide_diagonal=False, hide_threshold=None):
    """ Prints confusion matrixes in a 'pretty' way in the console output
    """
    columnwidth = max([len(x) for x in labels] + [5])  # 5 is value length
    empty_cell = " " * columnwidth
    # Print header
    print("    " + empty_cell, end=" ")
    for label in labels:
        print("%{0}s".format(columnwidth) % label, end=" ")
    print()
    # Print rows
    for i, label1 in enumerate(labels):
        print("    %{0}s".format(columnwidth) % label1, end=" ")
        for j in range(len(labels)):
            cell = "%{0}.1f".format(columnwidth) % cm[i, j]
            if hide_zeroes:
                cell = cell if float(cm[i, j]) != 0 else empty_cell
            if hide_diagonal:
                cell = cell if i != j else empty_cell
            if hide_threshold:
                cell = cell if cm[i, j] > hide_threshold else empty_cell
            print(cell, end=" ")
        print()
        
def evaluate_model(X, y, model):
    """ Evaluates a keras model given data and labels. Prints metrics 
        and confusion matrix to console and returns metrics as floats
    
        inputs: 
        X -- an [n,m] numpy array of observations 
        y -- an [n,1] numpy array of class labels
        model -- a loaded keras model object
        
        outputs:
        auroc -- Area under the receiver operating characteristic
        accuracy -- Proportion of correct/incorrect predictions
        sensitvity -- The true positive rate or recall  
        specificity -- The true negative rate 
    """
    y_pred = model.predict(X)
    y_pred_binary = (np.array(y_pred) > 0.5) * 1
    
    TN, FP, FN, TP = confusion_matrix(y, y_pred_binary).ravel()
    fpr, tpr, thresholds = metrics.roc_curve(y, y_pred, pos_label=1)
    
    auroc = metrics.auc(fpr, tpr)
    accuracy = accuracy_score(y_pred=y_pred_binary, y_true=y)
    sensitivity = TP / (TP + FN)
    specificity = TN / (TN + FP)
    
    print('acc:',accuracy)
    print('auc:',auroc)
    print('sensitivity:',sensitivity)
    print('specificity:',specificity)
    print('')
    cm = confusion_matrix(y, y_pred_binary)
    print_cm(cm,["Non-AHE","AHE"])
    return auroc, accuracy, sensitivity, specificity

Using TensorFlow backend.


In [3]:
# Load Data
X = np.load("/mnt/data2/brandon/reprocessed_data_apr1/physionet_data/apr16/normalized/test_X_obs60_lag10.npy")
y = np.load("/mnt/data2/brandon/reprocessed_data_apr1/physionet_data/apr16/normalized/test_y_obs60_lag10.npy")

In [4]:
# Load Model
model_filepath = '/mnt/data2/brandon/ScientificReports/LSTM_DeepEnsamble/SR_LSTM_base_model_lag10_obs60_1.hdf5'
model = load_model(model_filepath, custom_objects={'auc_roc':auc_roc})

In [5]:
# Evaluate Model 
auc, acc, sens, spec = evaluate_model(X, y, model)

acc: 0.8154651879235982
auc: 0.8497569010866388
sensitivity: 0.7368421052631579
specificity: 0.8188242852553806

            Non-AHE     AHE 
    Non-AHE  2549.0   564.0 
        AHE    35.0    98.0 
